In [1]:
import cv2

In [70]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os

In [ ]:
%qtconsole

In [ ]:
# list files in director

In [71]:
os.chdir("/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaFruits_28Sept2016/")

In [80]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [87]:
photoList = [ x for x in onlyfiles if "DSC" not in x and not x.startswith('.') and x.endswith('.jpg')]

In [88]:
# read in image as greyscale
kk = 0
fpth = photoList[kk]
img = cv2.imread(fpth,0)

In [89]:
io.imshow(img) # so fast, compared to sklearn

In [90]:
# Otsu's thresholding
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [91]:
# eroding and dilating
# way faster than skimage
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
erosion = cv2.erode(th3,kernel,iterations = 10)
dilation = cv2.dilate(erosion, kernel, iterations = 10)
plt.imshow(dilation, cmap = 'gray')

In [92]:
# find contours
#im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cts, im2 = cv2.findContours(dilation,  cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [93]:
# draw all contours (light)
plt.clf()
cv2.drawContours(img, cts, -1, (200,20,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [30]:
# draw particular contours
ii  = 2
cnt = cts[ii]
cv2.drawContours(img, [cnt], 0, (0,255,0), -1)
io.imshow(img)
    

In [49]:
# same as below, but puts it in a list
# for ii in range(len(cts)):
#     print cv2.arcLength(cts[ii], True) 

perimeters = [cv2.arcLength(cts[ii], True) for ii in range(len(cts))]


In [50]:
# calculate areas
areas = [cv2.contourArea(cts[ii], False) for ii in range(len(cts))]


# calculate circularity
circularity = (perimeters .^ 2) ./ (4 * pi * area)
If this is closer to 1, then the object is more circle-like

In [51]:
circularity = [(x **2) / (4*3.14159*y) for x,y in zip(perimeters, areas)]
circularity

[1.1260327330676922,
 1.2921066348438515,
 1.2085209546142601,
 1.1922025956869875,
 1.232210354229757,
 1.2832396200451643,
 1.2647052384386728]

In [55]:
arrs = np.array([i > 1.20 for i in circularity])

In [56]:
ctd = np.array(cts)
filt_contours = ctd[arrs]

In [57]:
# draw filtered contours (black)
plt.clf()
cv2.drawContours(img, filt_contours, -1, (0,255,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [58]:
# calculate diameters in pixels
# area = pi * r ^2
# r = sqrt(area / pi)

radii = np.array([np.sqrt(aa) for aa in areas])
radii

array([ 604.53453169,  103.89177061,  105.13324878,  101.81601053,
        114.0679622 ,  116.57615537,   91.82047702])

In [65]:
# show circle with largest diameter
bgCirc = np.array(radii == radii.max())

In [69]:
# draw filtered contours (black)
cv2.drawContours(img, ctd[bgCirc], -1, (40,255,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [ ]:
# blob detector
 
# Read image
im = img
 
# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector()
 
# Detect blobs.
keypoints = detector.detect(im)
 
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


In [ ]:
# Show keypoints
cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)

In [ ]:
w

In [ ]:
# plot all the images and their histograms
images = [ img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram',"Otsu's Thresholding",
         'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]

for i in xrange(2):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
contours, hierarchy = cv2.findContours(dilation,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)